In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# Load the pivot table containing node data
pivot_df = pd.read_csv('fifty_temperatures_motes.csv')
pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)

# Parameters
num_nodes = 50
delta_t = 1
beta_1 = 0.3
beta_2 = 0.3
M = 5
penalty = 0.5
L = 100  # Fairness constraint time limit

# Initialize variables
polling_counter = {i: 0 for i in range(1, num_nodes + 1)}
last_update_times = {i: 0 for i in range(1, num_nodes + 1)}
dewma_states_sink = {f'mote{i}': (20.0, 0.1) for i in range(1, num_nodes + 1)}

# Track only filtered values transmitted to the sink
transmitted_data = {i: [] for i in range(1, num_nodes + 1)}
original_data = {i: pivot_df[f'mote{i}'].tolist() for i in range(1, num_nodes + 1)}

# DEWMA update function
def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t):
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

# Poll nodes and collect data
def poll_nodes(pivot_df, current_time, M, penalty):
    whittle_indices = {}
    for mote in range(1, num_nodes + 1):
        last_update_time = last_update_times[mote]
        dynamic_delta_t = current_time - last_update_time
        last_state_value, last_rate_of_change = dewma_states_sink[f'mote{mote}']
        future_aoii_passive = dynamic_delta_t * last_rate_of_change
        whittle_indices[mote] = future_aoii_passive - penalty  # simplified Whittle index for polling decision

    # Select top M nodes to poll
    nodes_to_poll = sorted(whittle_indices, key=whittle_indices.get, reverse=True)[:M]

    # Poll selected nodes and collect filtered values at those points
    for mote in nodes_to_poll:
        column_name = f'mote{mote}'
        measured_value = pivot_df.loc[current_time, column_name]

        # Update DEWMA state at polling points only
        last_state_value, last_rate_of_change = dewma_states_sink[column_name]
        filtered_value, rate_of_change = update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t)
        
        # Save transmitted filtered data point
        transmitted_data[mote].append({'timestamp': current_time, 'filtered_value': filtered_value})
        
        # Update DEWMA state at the sink
        dewma_states_sink[column_name] = (filtered_value, rate_of_change)
        last_update_times[mote] = current_time

# Run the polling simulation
for t in range(1000):  # Using a smaller time frame for demonstration; adjust as needed
    poll_nodes(pivot_df, t, M, penalty)

# Plotting for a specific node
selected_mote = 5
timestamps = range(len(original_data[selected_mote]))

# Extract transmitted filtered values and timestamps for the selected mote
transmitted_timestamps = [data_point['timestamp'] for data_point in transmitted_data[selected_mote]]
transmitted_filtered_values = [data_point['filtered_value'] for data_point in transmitted_data[selected_mote]]

# Fit a spline to the transmitted filtered values
cs_filtered = CubicSpline(transmitted_timestamps, transmitted_filtered_values)
t_dense = np.linspace(min(transmitted_timestamps), max(transmitted_timestamps), 500)
filtered_spline = cs_filtered(t_dense)

# Plot the data
plt.figure(figsize=(10, 6))

# Original data across all time steps
plt.plot(timestamps, original_data[selected_mote], 'o', color='orange', markersize=5, label='Original Data', alpha=0.5)

# Filtered data points transmitted to the sink
plt.plot(transmitted_timestamps, transmitted_filtered_values, 'x', color='green', markersize=7, label='Transmitted Filtered Data', alpha=0.8)

# Spline fitted to the transmitted data points
plt.plot(t_dense, filtered_spline, 'b-', linewidth=2, label='Spline Fit - Transmitted Data')

# Customize plot
plt.xlabel('Time Step')
plt.ylabel('Value')
plt.legend(loc='upper right', fontsize=12)
plt.grid()
plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# Load dataset
pivot_df = pd.read_csv('top_fifty_motes_temperatures.csv')
pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)
pivot_df = pivot_df.head(10000)

# Parameters
num_nodes = 50                         # Total number of nodes
delta_t = 1                            # Time interval between updates (1 second)
num_nodes_to_poll = 10                 # Fixed number of nodes to poll
penalty_aoii = 0.5                     # Penalty for polling action, also used as threshold
gamma = 0.9                            # Discount factor for future rewards
beta_1 = 0.3                           # Smoothing factor for state value in dEWMA
beta_2 = 0.1                           # Smoothing factor for rate of change in dEWMA
pdr = 0.9                              # Probability of successful transmission

# Function to calculate AoII (Age of Incorrect Information)
def calculate_aoii(current_time, last_update_time, rate_of_change):
    return abs((current_time - last_update_time) * rate_of_change)

# Function to update state using dEWMA filtering
def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t):
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

# Function to predict state at the sink
def predict_sink_state(x1, x2, t, t_sink):
    delta_t = t_sink - t
    return x1 + x2 * delta_t

# Whittle-based simulation function for AoII-Whittle with polling count and state reproduction
def run_simulation_aoii_whittle(num_nodes_to_poll, delta_t, penalty_aoii):
    state_values = {i: 20.0 for i in range(1, num_nodes + 1)}           # Initial state values
    rate_of_change = {i: 0.01 for i in range(1, num_nodes + 1)}         # Initial rate of change values
    last_update_times = {i: 0 for i in range(1, num_nodes + 1)}         # Last update times
    polling_count = {i: 0 for i in range(1, num_nodes + 1)}             # Track number of times each node is polled
    sink_predictions = []  # Store sink predictions
    timestamps = []        # Store timestamps of predictions

    for idx, row in pivot_df.iterrows():
        current_time_step = idx

        # Update state continuously using dEWMA
        for mote in state_values:
            measured_value = row[f'mote{mote}']
            state_values[mote], rate_of_change[mote] = update_node_state_dewma(
                measured_value, state_values[mote], rate_of_change[mote], delta_t
            )

        # Calculate Whittle Index for each node
        whittle_arms = {}
        for mote in state_values:
            current_aoii = calculate_aoii(current_time_step, last_update_times[mote], rate_of_change[mote])
            
            # Future AoII values for active and passive actions
            future_aoii_active = 0 if np.random.rand() < pdr else current_aoii + delta_t * rate_of_change[mote]
            future_aoii_passive = current_aoii + delta_t * rate_of_change[mote]
            
            # Q-values for active and passive actions
            q_active = current_aoii + gamma * future_aoii_active + penalty_aoii
            q_passive = current_aoii + gamma * future_aoii_passive
            
            # Update Whittle index
            whittle_arms[mote] = q_passive - q_active

        # Select nodes where active action has a lower penalty than passive action
        nodes_to_poll = [
            mote for mote in whittle_arms if whittle_arms[mote] < 0
        ]

        # Sort and select top nodes with the highest Whittle index values
        nodes_to_poll = sorted(nodes_to_poll, key=lambda x: whittle_arms[x], reverse=True)[:num_nodes_to_poll]

        # Perform actions for nodes that are being polled
        for mote in nodes_to_poll:
            last_update_times[mote] = current_time_step
            polling_count[mote] += 1  # Increment polling count for polled nodes

            # Predict sink state for the polled node
            t_sink = current_time_step + 32  # Predict state 32 seconds into the future
            y_sink = predict_sink_state(state_values[mote], rate_of_change[mote], current_time_step, t_sink)

            sink_predictions.append(y_sink)
            timestamps.append(current_time_step)

    return sink_predictions, timestamps

# Run the simulation using Whittle AoII
sink_predictions, timestamps = run_simulation_aoii_whittle(num_nodes_to_poll, delta_t, penalty_aoii)

# Ensure timestamps are strictly increasing by sorting and removing duplicates
unique_timestamps, unique_indices = np.unique(timestamps, return_index=True)
unique_sink_predictions = np.array(sink_predictions)[unique_indices]

# Fit a spline to the sink predictions
if len(unique_timestamps) > 1:  # Check if there are enough unique points for spline fitting
    cs = CubicSpline(unique_timestamps, unique_sink_predictions)

    # Generate a dense set of time points for a smooth spline plot
    t_dense = np.linspace(min(unique_timestamps), max(unique_timestamps), 100)
    T_dense = cs(t_dense)

    # Convert the dense time points back to datetime for plotting
    t_dense_datetime = pd.to_datetime(t_dense, unit='s')

# Plot the actual sensor readings and predicted values at the sink
plt.figure(figsize=(12, 6))
plt.plot(pivot_df['mote1'].index, pivot_df['mote1'], label='Actual Data', marker='o')
if len(unique_timestamps) > 1:
    plt.plot(unique_timestamps, unique_sink_predictions, 'ro', label='Sink Predictions')
    plt.plot(t_dense_datetime, T_dense, 'b--', label='Spline Fit')
plt.xlabel('Time Steps')
plt.ylabel('Sensor Value')
plt.title('Sensor Data and Predictions at Sink (Whittle AoII)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("Total number of sink predictions:", len(sink_predictions))


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.interpolate import CubicSpline

# Load dataset
pivot_df = pd.read_csv('top_fifty_motes_temperatures.csv')
pivot_df = pivot_df.apply(lambda x: x.fillna(x.mean()), axis=0)
pivot_df = pivot_df.head(10000)

# Parameters
num_nodes = 50                         # Total number of nodes
delta_t = 1                            # Time interval between updates (1 second)
num_nodes_to_poll = 10                 # Fixed number of nodes to poll
penalty_aoii = 0.5                     # Penalty for polling action, also used as threshold
gamma = 0.9                            # Discount factor for future rewards
beta_1 = 0.3                           # Smoothing factor for state value in dEWMA
beta_2 = 0.1                           # Smoothing factor for rate of change in dEWMA
pdr = 0.9                              # Probability of successful transmission

# Function to calculate AoII (Age of Incorrect Information)
def calculate_aoii(current_time, last_update_time, rate_of_change):
    return abs((current_time - last_update_time) * rate_of_change)

# Function to update state using dEWMA filtering
def update_node_state_dewma(measured_value, last_state_value, last_rate_of_change, delta_t):
    x1 = beta_1 * measured_value + (1 - beta_1) * (last_state_value + last_rate_of_change * delta_t)
    x2 = beta_2 * (x1 - last_state_value) / delta_t + (1 - beta_2) * last_rate_of_change
    return x1, x2

# Function to predict state at the sink
def predict_sink_state(x1, x2, t, t_sink):
    delta_t = t_sink - t
    return x1 + x2 * delta_t

# Whittle-based simulation function for AoII-Whittle with polling count and state reproduction
def run_simulation_aoii_whittle(num_nodes_to_poll, delta_t, penalty_aoii):
    state_values = {i: 20.0 for i in range(1, num_nodes + 1)}           # Initial state values
    rate_of_change = {i: 0.01 for i in range(1, num_nodes + 1)}         # Initial rate of change values
    last_update_times = {i: 0 for i in range(1, num_nodes + 1)}         # Last update times
    polling_count = {i: 0 for i in range(1, num_nodes + 1)}             # Track number of times each node is polled
    sink_predictions = []  # Store sink predictions
    timestamps = []        # Store timestamps of predictions

    for idx, row in pivot_df.iterrows():
        current_time_step = idx

        # Update state continuously using dEWMA
        for mote in state_values:
            measured_value = row[f'mote{mote}']
            state_values[mote], rate_of_change[mote] = update_node_state_dewma(
                measured_value, state_values[mote], rate_of_change[mote], delta_t
            )

        # Calculate Whittle Index for each node
        whittle_arms = {}
        for mote in state_values:
            current_aoii = calculate_aoii(current_time_step, last_update_times[mote], rate_of_change[mote])
            
            # Future AoII values for active and passive actions
            future_aoii_active = 0 if np.random.rand() < pdr else current_aoii + delta_t * rate_of_change[mote]
            future_aoii_passive = current_aoii + delta_t * rate_of_change[mote]
            
            # Q-values for active and passive actions
            q_active = current_aoii + gamma * future_aoii_active + penalty_aoii
            q_passive = current_aoii + gamma * future_aoii_passive
            
            # Update Whittle index
            whittle_arms[mote] = q_passive - q_active

        # Select nodes where active action has a lower penalty than passive action
        nodes_to_poll = [
            mote for mote in whittle_arms if whittle_arms[mote] < 0
        ]

        # Sort and select top nodes with the highest Whittle index values
        nodes_to_poll = sorted(nodes_to_poll, key=lambda x: whittle_arms[x], reverse=True)[:num_nodes_to_poll]

        # Perform actions for nodes that are being polled
        for mote in nodes_to_poll:
            last_update_times[mote] = current_time_step
            polling_count[mote] += 1  # Increment polling count for polled nodes

            # Predict sink state for the polled node
            t_sink = current_time_step + 32  # Predict state 32 seconds into the future
            y_sink = predict_sink_state(state_values[mote], rate_of_change[mote], current_time_step, t_sink)

            sink_predictions.append(y_sink)
            timestamps.append(current_time_step)

    return sink_predictions, timestamps

# Run the simulation using Whittle AoII
sink_predictions, timestamps = run_simulation_aoii_whittle(num_nodes_to_poll, delta_t, penalty_aoii)

# Fit a spline to the sink predictions
if sink_predictions:
    cs = CubicSpline(timestamps, sink_predictions)

    # Generate a dense set of time points for a smooth spline plot
    t_dense = np.linspace(min(timestamps), max(timestamps), 100)
    T_dense = cs(t_dense)

    # Convert the dense time points back to datetime for plotting
    t_dense_datetime = pd.to_datetime(t_dense, unit='s')

# Plot the actual sensor readings and predicted values at the sink
plt.figure(figsize=(12, 6))
plt.plot(pivot_df['mote1'].index, pivot_df['mote1'], label='Actual Data', marker='o')
if sink_predictions:
    plt.plot(timestamps, sink_predictions, 'ro', label='Sink Predictions')
    plt.plot(t_dense_datetime, T_dense, 'b--', label='Spline Fit')
plt.xlabel('Time Steps')
plt.ylabel('Sensor Value')
plt.title('Sensor Data and Predictions at Sink (Whittle AoII)')
plt.legend()
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()

print("Total number of sink predictions:", len(sink_predictions))
